In [1]:
from langchain_chroma import Chroma
import os
from pyprojroot import here
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from dotenv import load_dotenv
from pprint import pprint
import httpx
load_dotenv()

True

**Load environment variables and configs**

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPEN_AI_API_KEY")

EMBEDDING_MODEL = "text-embedding-3-small"
VECTORDB_DIR = "data/airline_policy_vectordb"
K=2

**Load the vectorDB**

In [3]:
vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=OpenAIEmbeddings(model=EMBEDDING_MODEL)
)
print("Number of vectors in vectordb:",
      vectordb._collection.count(), "\n\n")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Number of vectors in vectordb: 0 




**Sample Query**

In [4]:
message = "What is the cancelation rule for a flight ticket at swiss airline policy?"

**Perform the vector Search**

In [5]:
docs = vectordb.similarity_search(message, k=K)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: ghp_miap****************************CpTD. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [58]:
docs

[Document(metadata={'page': 10, 'source': 'd:\\Github\\Advanced-QA-and-RAG-Series\\RAG-with-unstructured-data-and-SQLAgent\\data\\unstructured_docs\\swiss_faq.pdf'}, page_content="cancel your Swiss Airlines flight within 24 hours, you can do so easily online. Just remember to check the terms and\nconditions of your ticket to make sure you're eligible for a refund.\nSwiss Air Cancellation Fees The cancellation fees for Swiss Air flights may vary depending on the type of ticket you have\npurchased. The airline of fers three dif ferent types of tickets, which are:\nFlex T icket - This ticket of fers the most flexibility and allows passengers to make changes or cancel their flight without\nincurring any additional fees. Standard T icket - This ticket of fers some flexibility but charges a fee for changes and\ncancellations. The fee for cancelling a standard ticket can range from 100 to 250 CHF , depending on the route and time\nof cancellation. Economy T icket - This ticket of fers the lea

**Prepare the prompt for the GPT model**

In [6]:
question = "# User new question:\n" + message
retrieved_content = ""
for doc in docs:
    retrieved_content += f"{doc.page_content}\n\n"
prompt = f"# Content:\n{retrieved_content}\n\n{question}"

NameError: name 'docs' is not defined

Prepared prompt

In [60]:
pprint(prompt)

('# Content:\n'
 'cancel your Swiss Airlines flight within 24 hours, you can do so easily '
 'online. Just remember to check the terms and\n'
 "conditions of your ticket to make sure you're eligible for a refund.\n"
 'Swiss Air Cancellation Fees The cancellation fees for Swiss Air flights may '
 'vary depending on the type of ticket you have\n'
 'purchased. The airline of fers three dif ferent types of tickets, which '
 'are:\n'
 'Flex T icket - This ticket of fers the most flexibility and allows '
 'passengers to make changes or cancel their flight without\n'
 'incurring any additional fees. Standard T icket - This ticket of fers some '
 'flexibility but charges a fee for changes and\n'
 'cancellations. The fee for cancelling a standard ticket can range from 100 '
 'to 250 CHF , depending on the route and time\n'
 'of cancellation. Economy T icket - This ticket of fers the least flexibility '
 'and charges a fee for any changes or cancellations.\n'
 'The fee for cancelling an economy 

**Pass the prompt to the GPT model and get the response**

In [ ]:
client = OpenAI(
    base_url="https://models.github.ai/inference",
    api_key=os.getenv("OPEN_AI_API_KEY"),
    http_client=httpx.Client()
)

response = client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You will receive a user's query and possible content where the answer might be. If the answer is found, provide it, if not, state that the answer does not exist."},
        {"role": "user", "content": "How do you like the weather today?"},  # Example user message
    ]
)

The answer does not exist.


Printing the response

In [9]:
pprint(response.choices[0].message.content)

'The answer does not exist.'


**RAG Tool design using LangChain**

In [63]:
from langchain_core.tools import tool

@tool
def lookup_swiss_airline_policy(query: str)->str:
    """Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query."""
    vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=OpenAIEmbeddings(model=EMBEDDING_MODEL)
    )
    docs = vectordb.similarity_search(query, k=K)
    return "\n\n".join([doc.page_content for doc in docs])

In [64]:
print(lookup_swiss_airline_policy.name)
print(lookup_swiss_airline_policy.args)
print(lookup_swiss_airline_policy.description)

lookup_swiss_airline_policy
{'query': {'title': 'Query', 'type': 'string'}}
Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query.


In [67]:
pprint(lookup_swiss_airline_policy.invoke("can I cancel my ticket?"))

('cancel your Swiss Airlines flight within 24 hours, you can do so easily '
 'online. Just remember to check the terms and\n'
 "conditions of your ticket to make sure you're eligible for a refund.\n"
 'Swiss Air Cancellation Fees The cancellation fees for Swiss Air flights may '
 'vary depending on the type of ticket you have\n'
 'purchased. The airline of fers three dif ferent types of tickets, which '
 'are:\n'
 'Flex T icket - This ticket of fers the most flexibility and allows '
 'passengers to make changes or cancel their flight without\n'
 'incurring any additional fees. Standard T icket - This ticket of fers some '
 'flexibility but charges a fee for changes and\n'
 'cancellations. The fee for cancelling a standard ticket can range from 100 '
 'to 250 CHF , depending on the route and time\n'
 'of cancellation. Economy T icket - This ticket of fers the least flexibility '
 'and charges a fee for any changes or cancellations.\n'
 'The fee for cancelling an economy ticket can range